In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
# Tensorflow가 활용할 GPU가 장착되어 있는지 확인해 봅니다.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### 1) ResNet 기본 블록 구성하기

In [61]:
def build_resnet34_block(
    input_layer,
    num_cnn=3, 
    channel=64,
    block_num=1,
    is_plain=False
):
    # 입력 레이어
    x = input_layer

    ## CNN 레이어
    for cnn_num in range(num_cnn):
        identity = x
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_1_conv{cnn_num}'
        )(x)
        x = keras.layers.BatchNormalization(
            name=f'block{block_num}_1_batch_normal{cnn_num}'
        )(x)
        x = keras.layers.Activation(
            "relu", 
            name=f'block{block_num}_1_activation{cnn_num}'
        )(x)
        
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_2_conv{cnn_num}'
        )(x)
        x = keras.layers.BatchNormalization(
            name=f'block{block_num}_2_batch_normal{cnn_num}'
        )(x)
        
        if not is_plain:
            ## skip connection의 채널 수를 맞춰준다. 
            if identity.shape[-1] != x.shape[-1]:
                identity = keras.layers.Conv2D(
                    filters=x.shape[-1], 
                    kernel_size=(1, 1), 
                    strides=(1, 1), 
                    padding='same'
                )(identity)

            x = keras.layers.Add()([x,identity])
        
        x = keras.layers.Activation(
            "relu", 
            name=f'block{block_num}_2_activation{cnn_num}'
        )(x)        
        
    #     Max Pooling 레이어
    # 마지막 블록 뒤에는 pooling을 하지 않음
    if cnn_num != (num_cnn-1):        
        x = keras.layers.MaxPooling2D(
            pool_size=(2, 2),
            strides=2,
            name=f'block{block_num}_pooling'
        )(x)

    return x

In [66]:
def build_resnet50_block(
    input_layer,
    num_cnn=3, 
    channel=64,
    block_num=1,
    is_plain=False
):
    # 입력 레이어
    x = input_layer

    ## CNN 레이어
    for cnn_num in range(num_cnn):
        identity = x
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(1,1),
            kernel_initializer='he_normal',
#             padding='same',
            name=f'block{block_num}_1_conv{cnn_num}'
        )(x)
        x = keras.layers.BatchNormalization(
            name=f'block{block_num}_1_batch_normal{cnn_num}'
        )(x)
        x = keras.layers.Activation(
            "relu", 
            name=f'block{block_num}_1_activation{cnn_num}'
        )(x)
        
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_2_conv{cnn_num}'
        )(x)
        x = keras.layers.BatchNormalization(
            name=f'block{block_num}_2_batch_normal{cnn_num}'
        )(x)
        x = keras.layers.Activation(
            "relu", 
            name=f'block{block_num}_2_activation{cnn_num}'
        )(x)
        
        if num_cnn == 0:
            identity = keras.layers.Conv2D(
                filters=channel*4,
                kernel_size=(1,1),
                kernel_initializer='he_normal',
                padding='same',
                name=f'block{block_num}_0_conv{cnn_num}'
            )(identity)
        
        x = keras.layers.Conv2D(
            filters=channel*4,
            kernel_size=(1,1),
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_3_conv{cnn_num}'
        )(x)
        x = keras.layers.BatchNormalization(
            name=f'block{block_num}_3_batch_normal{cnn_num}'
        )(x)

        if not is_plain:
            ## skip connection의 채널 수를 맞춰준다. 
            if identity.shape[-1] != x.shape[-1]:
                identity = keras.layers.Conv2D(filters=x.shape[-1], kernel_size=(1, 1), strides=(1, 1), padding='same')(identity)

            x = keras.layers.Add()([x,identity])
        else:
            pass
        
        if cnn_num != (num_cnn - 1):        
            x = keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                name=f'block{block_num}_pooling{cnn_num}'
            )(x)
        
        
    return x

In [67]:
# ResNet 모델 자체를 생성하는 함수입니다.
def build_resnet(
    input_shape=(224,224,3),
    num_cnn_list=[3,4,6,3],  # ResNet-34 혹은 ResNet-50 구조에서 사용할 CNN 개수
    channel_list=[64,128,256,512],   # 각 블록의 채널 크기
    num_classes=2,   # 분류할 클래스 수
    is_50=False,      # False면 ResNet-34, True면 ResNet-50
    is_plain=False    # Plain 네트워크 여부 (skip connection을 사용하지 않는지 여부)
):
    
    assert len(num_cnn_list) == len(channel_list) #모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    output = input_layer
    
    ### Conv1
    output = keras.layers.Conv2D(
        filters=64,
        kernel_size = (7,7),
        strides = 2,
        padding = 'valid')(output)
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation("relu")(output)
    output = keras.layers.MaxPooling2D(padding="same")(output)
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    if not is_50:
        build_resnet_block = build_resnet34_block
    else: 
        build_resnet_block = build_resnet50_block
        
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_resnet_block(
            output,
            num_cnn=num_cnn, 
            channel=channel,
            block_num=i, 
            is_plain=is_plain
        )
    
    output = keras.layers.AveragePooling2D(padding="same")(output)
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(512, activation='relu', name='fc1000')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

### 2) ResNet-34, ResNet-50 Complete Model

In [68]:
resnet34_model = build_resnet(input_shape=(224,224,3), is_50=False, is_plain=False)
plain34_model = build_resnet(input_shape=(224,224,3), is_50=False, is_plain=True)

In [69]:
resnet34_model.summary()

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 109, 109, 64) 9472        input_23[0][0]                   
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 109, 109, 64) 256         conv2d_46[0][0]                  
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 109, 109, 64) 0           batch_normalization_22[0][0]     
___________________________________________________________________________________________

In [6]:
resnet50_model = build_resnet(input_shape=(224,224,3), is_50=True, is_plain=False)
plain50_model = build_resnet(input_shape=(224,224,3), is_50=True, is_plain=True)

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

In [ ]:
resnet34_model.summary()

In [ ]:
resnet50_model.summary()

### 3) 일반 네트워크(plain network) 만들기

In [ ]:
plain34_model.summary()

In [ ]:
plain50_model.summary()

### 4) ResNet-50 vs Plain-50 또는 ResNet-34 vs Plain-34

In [115]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

datasets, info = tfds.load('cats_vs_dogs', 
                     split=['train[:80%]', 'train[80%:]'], 
                     with_info=True, 
                     as_supervised=True)

In [143]:
train_dataset, test_dataset = datasets

In [144]:
def preprocess_image(image, label):
    image = tf.image.resize(image, [224,224])  # 이미지 크기 조정
    image = image / 255.0  # 정규화
    return image, label

In [145]:
# 전처리 적용
train_dataset = train_dataset.map(preprocess_image).batch(16).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess_image).batch(16).prefetch(tf.data.AUTOTUNE)

In [146]:
# 이미지와 라벨 구분
for images, labels in train_dataset.take(1):  # 첫 배치 가져오기
    # 이미지와 라벨 확인
    print("Images shape:", images.shape)  # 이미지 텐서의 shape
    print("Labels shape:", labels.shape)  # 라벨 텐서의 shape

Images shape: (64, 224, 224, 3)
Labels shape: (64,)


In [148]:
## 모델 complie
resnet34_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)
plain34_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)
resnet50_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)
plain50_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)


/opt/conda/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [149]:
EPOCH = 5

In [151]:
history_resnet34 = resnet34_model.fit(
    train_dataset,
#     steps_per_epoch=int(train_dataset.splits['train'].num_examples/BATCH_SIZE),
#     validation_steps=int(train_dataset.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=test_dataset,
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/5


ResourceExhaustedError:  OOM when allocating tensor with shape[64,512,56,56] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_26/block3_2_conv0/Conv2D (defined at tmp/ipykernel_37/2386665842.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_209252]

Function call stack:
train_function


In [ ]:
history_plain34 = plain34_model.fit(
    train_dataset,
#     steps_per_epoch=int(train_dataset.splits['train'].num_examples/BATCH_SIZE),
#     validation_steps=int(train_dataset.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=test_dataset,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_resnet34.history['loss'], 'r')
plt.plot(history_plain34.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['vgg_16', 'vgg_19'], loc='upper left')
plt.show()